> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [ ]:
import pandas as pd

df = pd.read_csv('salaries.csv')

print(df.info())
display(df.head())

print(f"SEX: {df['SEX'].unique()}")
print(f"DESIGNATION: {df['DESIGNATION'].unique()}")
print(f"UNIT: {df['UNIT'].unique()}")

#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [ ]:
### Escreva sua resposta aqui

from sqlalchemy import Column, Integer, String, Float, Date, Enum
from sqlalchemy.orm import declarative_base
import enum

Base = declarative_base()

class SexEnum(enum.Enum):
    F = 'F'
    M = 'M'

class DesignationEnum(enum.Enum):
    Analyst = 'Analyst'
    Associate = 'Associate'
    Senior_Analyst = 'Senior Analyst'
    Senior_Manager = 'Senior Manager'
    Manager = 'Manager'
    Director = 'Director'

class UnitEnum(enum.Enum):
    Finance = 'Finance'
    Web = 'Web'
    IT = 'IT'
    Operations = 'Operations'
    Marketing = 'Marketing'
    Management = 'Management'

class Professional(Base):
    __tablename__ = 'profissionais'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String)
    last_name = Column(String)
    sex = Column(Enum(SexEnum))
    doj = Column(Date)
    current_date = Column(Date)
    designation = Column(Enum(DesignationEnum))
    age = Column(Integer)
    salary = Column(Float)
    unit = Column(Enum(UnitEnum))
    leaves_used = Column(Integer)
    leaves_remaining = Column(Integer)
    ratings = Column(Float)
    past_exp = Column(Float)

#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [ ]:
### Escreva sua resposta aqui

from sqlalchemy import create_engine
engine = create_engine('sqlite:///salarios.db')

#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [ ]:
### Escreva sua resposta aqui

Base.metadata.create_all(engine)

#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [ ]:
### Escreva sua resposta aqui

df_db = df.copy()
df_db['DOJ'] = pd.to_datetime(df_db['DOJ']).dt.date
df_db['CURRENT DATE'] = pd.to_datetime(df_db['CURRENT DATE']).dt.date

# Renomeando colunas
mapping = {
    'FIRST NAME': 'first_name',
    'LAST NAME': 'last_name',
    'SEX': 'sex',
    'DOJ': 'doj',
    'CURRENT DATE': 'current_date',
    'DESIGNATION': 'designation',
    'AGE': 'age',
    'SALARY': 'salary',
    'UNIT': 'unit',
    'LEAVES USED': 'leaves_used',
    'LEAVES REMAINING': 'leaves_remaining',
    'RATINGS': 'ratings',
    'PAST EXP': 'past_exp'
}
df_db = df_db.rename(columns=mapping)

# Coloca no SQLite
df_db.to_sql('profissionais', con=engine, if_exists='append', index=False)

#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [ ]:
### Execute aqui sua query SQL com SQLAlchemy

from sqlalchemy import text

with engine.connect() as conn:
    query = text("""
        SELECT designation, 
               MIN(salary / 12) as min_mensal, 
               MAX(salary / 12) as max_mensal, 
               AVG(salary / 12) as avg_mensal
        FROM profissionais
        GROUP BY designation
    """)
    result = conn.execute(query)
    for row in result:
        print(row)

In [ ]:
### Execute aqui sua query SQL com SQLAlchemy + Pandas

with engine.connect() as conn:
    df_result = pd.read_sql_query(query, conn)
    print(df_result)

In [ ]:
### Execute aqui sua query com SQLAlchemy ORM

from sqlalchemy.orm import Session
from sqlalchemy import select, func

with Session(engine) as session:
    stmt = select(
        Professional.designation,
        func.min(Professional.salary / 12).label('min_mensal'),
        func.max(Professional.salary / 12).label('max_mensal'),
        func.avg(Professional.salary / 12).label('avg_mensal')
    ).group_by(Professional.designation)
    
    results = session.execute(stmt).all()
    for r in results:
        print(f"Cargo: {r.designation.value} | Média Mensal: {r.avg_mensal:.2f}")